In [6]:
"""
Скрипт для подсчета показателей трудоустройства выпускников
"""

import pandas as pd
import os
import openpyxl
import math


def check_data(cell):
    """
    Метод для проверки значения ячейки
    :param cell: значение ячейки
    :return: число в формате int
    """
    # Проверям на строку
    if type(cell) == str:
        return 0
    # Проверяем на пустую ячейку
    if math.isnan(cell):
        return 0
    if type(cell) == int:
        return cell
    if type(cell) == float:
        return int(cell)
    else:
        return 0


def create_low_dct():
    """
    Функция для создания словаря низкого уровня вида 8 ключей:{23 ключа:0}
    :return: Словарь нужной структуры
    """
    # Создаем базовые структы данных
    # Список категорий по которым будет идти подсчет

    # ахаха особенности оформления документа что ключи придется писать так как они там написаны т.е. вот с такими пробелами
    # base_cat = ['Всего ', 'Лица с ограниченными возможностями здоровья',
    #             '           из них (из строки 02): инвалиды и дети-инвалиды',
    #             'Инвалиды и дети-инвалиды (кроме учтенных в строке 03)',
    #             'Имеют договор о целевом обучении',
    #             '           из них (из строки 05): Лица с ограниченными возможностями здоровья (имеющие договор о целевом обучении)',
    #             '                      из строки 06: инвалиды и дети-инвалиды (имеющие договор о целевом обучении)',
    #             '           из строки 05 инвалиды и дети-инвалиды (кроме учтенных в строке 07) (имеющие договор о целевом обучении)']
    base_cat = ['Всего', 'Лица с ограниченными возможностями здоровья',
                'из них (из строки 02): инвалиды и дети-инвалиды',
                'Инвалиды и дети-инвалиды (кроме учтенных в строке 03)',
                'Имеют договор о целевом обучении',
                'из них (из строки 05): Лица с ограниченными возможностями здоровья (имеющие договор о целевом обучении)',
                'из строки 06: инвалиды и дети-инвалиды (имеющие договор о целевом обучении)',
                'из строки 05 инвалиды и дети-инвалиды (кроме учтенных в строке 07) (имеющие договор о целевом обучении)']

    # Вот это словарь жутковато выглядит.Словарь показателей
    base_dct = {'Выпуск в 2021': 0, 'Трудоустроено человек': 0, 'Индивидуальные предприниматели': 0, 'Самозанятые': 0,
                'Призваны в Вооруженные силы': 0, 'Продолжили обучение': 0,
                'Находятся в отпуске по уходу за ребенком': 0,
                'Находящиеся под риском нетрудоустройства ': 0,
                'в том числе (из гр. 22): состоят на учете в центрах занятости в качестве ищущих работу или безработных': 0,
                'Прочее: смерть, переезд за пределы Российской Федерации, семейные обстоятельства, по состоянию здоровья и др.***': 0,
                'перечислить причины, указав число человек': '',
                'Не определились (ожидают результатов приемной кампании, ожидают призыва, находятся в активном поиске работы, собирают документы для открытия ИП. Выпускники временно не заняты, но их занятости ничего не угрожает)': 0,
                'Прогноз Трудоустройство': 0, 'Прогноз Индивидуальные предприниматели': 0, 'Прогноз Самозанятые': 0,
                'Прогноз Продолжили обучение': 0, 'Прогноз Призваны в Вооруженные силы': 0,
                'Прогноз Находятся в отпуске по уходу за ребенком': 0,
                'Прогноз Находящиеся под риском нетрудоустройства выпускники': 0,
                'Прогноз в том числе (из гр. 42): состоят на учете в центрах занятости в качестве ищущих работу или безработных': 0,
                'Прогноз Прочее: смерть, переезд за пределы Российской Федерации, семейные обстоятельства, по состоянию здоровья и др.*** ': 0,
                'Прогноз перечислить причины': '',
                'Причины, по которым выпускники находятся под риском нетрудоустройства, и принимаемые меры (тезисно)': ''}

    # Создаем словарь нижнего уровня
    # ССЫЛОЧНАЯ МОДЕЛЬ!!! Я ССЫЛАЛСЯ НА ОДИН И ТОТ ЖЕ СЛОВАРЬ УУУУ
    temp_dct = {key: base_dct.copy() for key in base_cat}
    return temp_dct


def calculation_data(dct, unique_code_lst):
    """
    Функция для подсчета получившихся значений для каждой специальности по всем ПОО.
    :param dct: словарь в котором находятся все данные по трудоустройству
            uniqie_code_lst: список всех уникальных специальностей
    :return: новый словарь где главным ключом будет являться код специальности.
    """
    # Создаем словарь где ключами верхнего уровня будут выступать коды специальности
    calcul_dct = {code: create_low_dct() for code in unique_code_spec}

    # Итерируемся по верхнему уровню словаря
    for poo, spec in dct.items():
        # Итерируемся по вложенному словарю вида {код специальности:{данные}}
        for cod_spec, data in spec.items():

            # Итерируемся по базовому словарю
            for cat, papam_dct in data.items():

                # Итерируемся по словарю параметров. Ха четверная итерация
                for param, value in papam_dct.items():
                    # Заполняем итоговый словарь
                    calcul_dct[cod_spec][cat][param] += value

    return calcul_dct


def check_data_text(cell):
    """
    Функция для обработки ячеек в которых могут встретиться как текстовые так и числовые данные. Колонки 25 и 45 как пример
    :param cell: значение ячейки
    :return:
    """

    if type(cell) == str:
        if cell !='nan':
            return f'{cell};'
        else:
            return ' ;'
    if type(cell) == float:
        if cell == 0.0:
            return f' ;'
    if type(cell) == int:
        return f'{str(cell)};'
    else:
        return ' ;'


# Открываем лист со списком СПО, с опцией для чтения чтобы ускорить процесс
temp = openpyxl.load_workbook('data/SPO.xlsx', read_only=True)
# Получаем список листов
temp_sheets = temp.sheetnames
# Очищаем список от лишних листов
# Создаем кортеж с названиями листов которые не будем обрабатывать
del_sheets = ('Форма 1', 'Форма 2', 'Коды и наименования программ','СВОД общий')

sheets = [sheet for sheet in temp_sheets if sheet not in del_sheets]
# Закрываем лист после чтения
temp.close()

# Создаем словарь верхнего уровня

high_dct = dict.fromkeys(sheets, dict())

# Создаем  общий список для специальностей
all_code_spec = []

# Создаем базовый датафрейм для проверки корректности заполнения
# columns = ['Наименование образовательной организации','Ведомственная принадлежность','Тип',
#            'Код профессии, специальности','Номер строки','Наименование показателей(категория выпускников)','Выпуск в 2021 году',
#            'Трудоустройство человек','Трудоустройство %','Индивидуальные предприниматели человек','Индивидуальные предприниматели %',
#            'Самозанятые человек','Самозанятые %','Призваны в Вооруженные Силы  человек','Призваны в Вооруженные Силы %',
#            'Продолжили обучение человек','Продолжили обучение %','Находятся в отпуске по уходу за ребенком человек','Находятся в отпуске по уходу за ребенком %',
#            'Находящиеся под риском нетрудоустройства человек','Находящиеся под риском нетрудоустройства %','в том числе (из гр. 22): состоят на учете в центрах занятости в качестве ищущих работу или безработных человек',
#            'в том числе (из гр. 22): состоят на учете в центрах занятости в качестве ищущих работу или безработных %',
#            'Прочее: смерть, переезд за пределы Российской Федерации, семейные обстоятельства, по состоянию здоровья и др.*** человек','Прочее: смерть, переезд за пределы Российской Федерации, семейные обстоятельства, по состоянию здоровья и др.*** %'
#     ,'Не определились (ожидают результатов приемной кампании, ожидают призыва, находятся в активном поиске работы, собирают документы для открытия ИП. Выпускники временно не заняты, но их занятости ничего не угрожает) человек'
#     ,'Не определились (ожидают результатов приемной кампании, ожидают призыва, находятся в активном поиске работы, собирают документы для открытия ИП. Выпускники временно не заняты, но их занятости ничего не угрожает) %',
#            'Прогноз Трудоустройство человек','Прогноз Трудоустройство %','Прогноз Индивидуальные предприниматели человек','Прогноз Индивидуальные предприниматели %',
#            'Прогноз Самозанятые человек','Прогноз Самозанятые %','Прогноз Продолжили обучение человек','Прогноз Продолжили обучение %',
#            'Прогноз Призваны в Вооруженные Силы человек','Прогноз Призваны в Вооруженные Силы %','Прогноз Находятся в отпуске по уходу за ребенком человек','Прогноз Находятся в отпуске по уходу за ребенком %',
#            'Прогноз Находящиеся под риском нетрудоустройства выпускники человек','Прогноз Находящиеся под риском нетрудоустройства выпускники %',
#            'Прогноз в том числе (из гр. 42): состоят на учете в центрах занятости в качестве ищущих работу или безработных человек','Прогноз в том числе (из гр. 42): состоят на учете в центрах занятости в качестве ищущих работу или безработных %',
#            'Прогноз Прочее: смерть, переезд за пределы Российской Федерации, семейные обстоятельства, по состоянию здоровья и др.*** человек','Прогноз Прочее: смерть, переезд за пределы Российской Федерации, семейные обстоятельства, по состоянию здоровья и др.*** %',
#            'Прогноз перечислить причины','Причины, по которым выпускники находятся под риском нетрудоустройства, и принимаемые меры (тезисно)',
#            '1','2','3','4',]
columns = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24',
       '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36',
       '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48',
       '49', '50']
base_correct_df = pd.DataFrame(columns=columns)


# Открываем файл с указанным листом и сохраняем его в датафрейм пропуская  первые 8 строк
for sheet in sheets:
    print(sheet)

    df = pd.read_excel('data/SPO.xlsx', sheet_name=sheet, skiprows=8, dtype={'Код профессии, специальности*': str})

    # Добавляем полученный датафрейм в базовый
    # base_correct_df = pd.concat([base_correct_df,df],ignore_index=True)
    base_correct_df = base_correct_df.append(df,ignore_index=True)


    # Создаем список специальность которые есть на данном листе
    raw_codes_spec = df['04'].unique()

    # Очищаем получившиеся коды от nan
    codes_spec = [code for code in raw_codes_spec if str(code) != 'nan']
    # Очищаем коды от пустых строк
    codes_spec = [code for code in codes_spec if code != ' ']

    # Добавляем полученный список в общий список специальностей
    all_code_spec.extend(codes_spec)

    # Создаем словарь через comprehnsions
    # Отбираем только строковые, отбрасывая ключ nan. В качестве значения присваиваем копию словаря низкого уровня
    # Забыл про ссылочную модель и не мог долго понять почему значения дублируются
    spec_code_dct = {code: create_low_dct() for code in codes_spec if type(code) == str}

    # Присваиваем полученный словарь со специальностями словарю high_dct
    """
    В итоге получается такая струтура {'КТИНЗ': {'29.01.07': {}, '35.01.23': {}, '43.01.09': {}, '54.01.06': {}, '54.01.03': {}, '35.02.07': {}, '54.02.02': {}}, 'ТСГХ': {}, 'БФКК': {}}
    """
    high_dct[sheet] = spec_code_dct
    # Итерируемся по полученному датафрейму через itertuples
    for row in df.itertuples():
        # Получаем код специальности для итерируемой строки. Отбрасываем пустые строки, просто проверяя есть ли такой ключ в главном словаре
        spec_code = row[4]
        if spec_code in high_dct[sheet]:
            # КТИНЗ:35.01.23:
            # print(high_dct[sheet][row_code])
            name_param = row[6].strip()
            high_dct[sheet][spec_code][name_param]['Выпуск в 2021'] = check_data(row[7])
            high_dct[sheet][spec_code][name_param]['Трудоустроено человек'] = check_data(row[8])
            high_dct[sheet][spec_code][name_param]['Индивидуальные предприниматели'] = check_data(row[10])
            high_dct[sheet][spec_code][name_param]['Самозанятые'] = check_data(row[12])
            high_dct[sheet][spec_code][name_param]['Призваны в Вооруженные силы'] = check_data(row[14])
            high_dct[sheet][spec_code][name_param]['Продолжили обучение'] = check_data(row[16])
            high_dct[sheet][spec_code][name_param]['Находятся в отпуске по уходу за ребенком'] = check_data(row[18])
            high_dct[sheet][spec_code][name_param]['Находящиеся под риском нетрудоустройства '] = check_data(row[20])
            high_dct[sheet][spec_code][name_param][
                'в том числе (из гр. 22): состоят на учете в центрах занятости в качестве ищущих работу или безработных'] = check_data(
                row[22])
            high_dct[sheet][spec_code][name_param][
                'Прочее: смерть, переезд за пределы Российской Федерации, семейные обстоятельства, по состоянию здоровья и др.***'] = check_data(
                row[23])
            high_dct[sheet][spec_code][name_param]['перечислить причины, указав число человек'] = check_data_text(row[25])
            high_dct[sheet][spec_code][name_param][
                'Не определились (ожидают результатов приемной кампании, ожидают призыва, находятся в активном поиске работы, собирают документы для открытия ИП. Выпускники временно не заняты, но их занятости ничего не угрожает)'] = check_data(
                row[26])
            high_dct[sheet][spec_code][name_param]['Прогноз Трудоустройство'] = check_data(row[28])
            high_dct[sheet][spec_code][name_param]['Прогноз Индивидуальные предприниматели'] = check_data(row[30])
            high_dct[sheet][spec_code][name_param]['Прогноз Самозанятые'] = check_data(row[32])
            high_dct[sheet][spec_code][name_param]['Прогноз Продолжили обучение'] = check_data(row[34])
            high_dct[sheet][spec_code][name_param]['Прогноз Призваны в Вооруженные силы'] = check_data(row[36])
            high_dct[sheet][spec_code][name_param]['Прогноз Находятся в отпуске по уходу за ребенком'] = check_data(
                row[38])
            high_dct[sheet][spec_code][name_param][
                'Прогноз Находящиеся под риском нетрудоустройства выпускники'] = check_data(row[40])
            high_dct[sheet][spec_code][name_param][
                'Прогноз в том числе (из гр. 42): состоят на учете в центрах занятости в качестве ищущих работу или безработных'] = check_data(
                row[42])
            high_dct[sheet][spec_code][name_param][
                'Прогноз Прочее: смерть, переезд за пределы Российской Федерации, семейные обстоятельства, по состоянию здоровья и др.*** '] = check_data(
                row[43])
            high_dct[sheet][spec_code][name_param]['Прогноз перечислить причины'] = check_data_text(row[45])
            high_dct[sheet][spec_code][name_param][
                'Причины, по которым выпускники находятся под риском нетрудоустройства, и принимаемые меры (тезисно)'] = check_data_text(row[46])
        else:
            continue

base_correct_df.to_excel('Общий.xlsx')
# Удаляем дубликаты
unique_code_spec = list(set(all_code_spec))
itog_dct = calculation_data(high_dct, unique_code_spec)

БРПК
БАК
БКТИС
БРТСиПТ
БРИТ
ЗАПТ
ДМТ
БРИЭТ
ГЭТ
КТИНЗ
БКН
БЛПК
БМК
АТ
БРМТИТ
РБМК
РМТ
ТСГХ
ПТ
БФКК
РМК
УУТЭТ
КИ им Чайк.
БРТАТ
БРХК
ББМК
СибГУТи
УУКЖТ
кВСГУТУ
кБГСХА
кБГУ


In [7]:
test_df=pd.DataFrame.from_dict(itog_dct,orient='index')

In [8]:
d = test_df.stack()
d

53.02.02  Всего                                                                                                      {'Выпуск в 2021': 4, 'Трудоустроено человек': ...
          Лица с ограниченными возможностями здоровья                                                                {'Выпуск в 2021': 0, 'Трудоустроено человек': ...
          из них (из строки 02): инвалиды и дети-инвалиды                                                            {'Выпуск в 2021': 0, 'Трудоустроено человек': ...
          Инвалиды и дети-инвалиды (кроме учтенных в строке 03)                                                      {'Выпуск в 2021': 0, 'Трудоустроено человек': ...
          Имеют договор о целевом обучении                                                                           {'Выпуск в 2021': 0, 'Трудоустроено человек': ...
                                                                                                                                           ...                       

In [ ]:
type(d)

In [9]:
frame = d.to_frame()
frame

0
53.02.02 Всего                                               {'Выпуск в 2021': 4, 'Трудоустроено человек': ...
         Лица с ограниченными возможностями здоровья         {'Выпуск в 2021': 0, 'Трудоустроено человек': ...
         из них (из строки 02): инвалиды и дети-инвалиды     {'Выпуск в 2021': 0, 'Трудоустроено человек': ...
         Инвалиды и дети-инвалиды (кроме учтенных в стро...  {'Выпуск в 2021': 0, 'Трудоустроено человек': ...
         Имеют договор о целевом обучении                    {'Выпуск в 2021': 0, 'Трудоустроено человек': ...
...                                                                                                        ...
38.02.06 Инвалиды и дети-инвалиды (кроме учтенных в стро...  {'Выпуск в 2021': 0, 'Трудоустроено человек': ...
         Имеют договор о целевом обучении                    {'Выпуск в 2021': 0, 'Трудоустроено человек': ...
         из них (из строки 05): Лица с ограниченными воз...  {'Выпуск в 2021': 0, 'Трудоустроено человек': ...
         из строки 06: инвалиды и дети-инвалиды (имеющие...  {'Выпуск в 2021': 0, 'Трудоустроено человек': ...
         из строки 05 инвалиды и дети-инвалиды (кроме уч...  {'Выпуск в 2021': 0, 'Трудоустроено человек': ...

[968 rows x 1 columns]

In [ ]:
frame['Выпуск в 2021'] =  frame[0].apply(lambda x:x.get('Выпуск в 2021'))
frame['Трудоустроено человек'] =  frame[0].apply(lambda x:x.get('Трудоустроено человек'))
frame['Индивидуальные предприниматели'] =  frame[0].apply(lambda x:x.get('Индивидуальные предприниматели'))
frame['Самозанятые'] =  frame[0].apply(lambda x:x.get('Самозанятые'))
frame['Призваны в Вооруженные силы'] =  frame[0].apply(lambda x:x.get('Продолжили обучение'))
frame['Находятся в отпуске по уходу за ребенком'] =  frame[0].apply(lambda x:x.get('Находятся в отпуске по уходу за ребенком'))
frame['Находящиеся под риском нетрудоустройства '] =  frame[0].apply(lambda x:x.get('Находящиеся под риском нетрудоустройства '))
frame['в том числе (из гр. 22): состоят на учете в центрах занятости в качестве ищущих работу или безработных'] =  frame[0].apply(lambda x:x.get('в том числе (из гр. 22): состоят на учете в центрах занятости в качестве ищущих работу или безработных'))
frame['Прочее: смерть, переезд за пределы Российской Федерации, семейные обстоятельства, по состоянию здоровья и др.***'] =  frame[0].apply(lambda x:x.get('Прочее: смерть, переезд за пределы Российской Федерации, семейные обстоятельства, по состоянию здоровья и др.***'))
frame['перечислить причины, указав число человек'] =  frame[0].apply(lambda x:x.get('перечислить причины, указав число человек'))
frame['Не определились (ожидают результатов приемной кампании, ожидают призыва, находятся в активном поиске работы, собирают документы для открытия ИП. Выпускники временно не заняты, но их занятости ничего не угрожает)'] =  frame[0].apply(lambda x:x.get('Не определились (ожидают результатов приемной кампании, ожидают призыва, находятся в активном поиске работы, собирают документы для открытия ИП. Выпускники временно не заняты, но их занятости ничего не угрожает)'))
frame['Прогноз Трудоустройство'] =  frame[0].apply(lambda x:x.get('Прогноз Трудоустройство'))
frame['Прогноз Индивидуальные предприниматели'] =  frame[0].apply(lambda x:x.get('Прогноз Индивидуальные предприниматели'))
frame['Прогноз Самозанятые'] =  frame[0].apply(lambda x:x.get('Прогноз Самозанятые'))
frame['Прогноз Продолжили обучение'] =  frame[0].apply(lambda x:x.get('Прогноз Продолжили обучение'))
frame['Прогноз Призваны в Вооруженные силы'] =  frame[0].apply(lambda x:x.get('Прогноз Призваны в Вооруженные силы'))
frame['Прогноз Находятся в отпуске по уходу за ребенком'] =  frame[0].apply(lambda x:x.get('Прогноз Находятся в отпуске по уходу за ребенком'))
frame['Прогноз Находящиеся под риском нетрудоустройства выпускники'] =  frame[0].apply(lambda x:x.get('Прогноз Находящиеся под риском нетрудоустройства выпускники'))
frame['Прогноз в том числе (из гр. 42): состоят на учете в центрах занятости в качестве ищущих работу или безработных'] =  frame[0].apply(lambda x:x.get('Прогноз в том числе (из гр. 42): состоят на учете в центрах занятости в качестве ищущих работу или безработных'))
frame['Прогноз Прочее: смерть, переезд за пределы Российской Федерации, семейные обстоятельства, по состоянию здоровья и др.*** '] =  frame[0].apply(lambda x:x.get('Прогноз Прочее: смерть, переезд за пределы Российской Федерации, семейные обстоятельства, по состоянию здоровья и др.*** '))
frame['Прогноз перечислить причины'] =  frame[0].apply(lambda x:x.get('Прогноз перечислить причины'))
frame['Причины, по которым выпускники находятся под риском нетрудоустройства, и принимаемые меры (тезисно)'] =  frame[0].apply(lambda x:x.get('Причины, по которым выпускники находятся под риском нетрудоустройства, и принимаемые меры (тезисно)'))


In [ ]:
frame

In [ ]:
frame = frame.drop([0],axis=1)
frame